In [1]:
!pip install selenium python-dotenv

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
from dotenv import load_dotenv
from selenium.webdriver.support.ui import Select

In [3]:
download_folder = os.path.join(os.getcwd(), 'files')
if not os.path.exists(download_folder):
    os.makedirs(download_folder)

# Set up Chrome options to automatically download files to the 'files' folder
chrome_options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": download_folder,  # Set the download directory
    "download.prompt_for_download": False,  # Disable the prompt for download
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True  # Enable safe browsing
}
chrome_options.add_experimental_option("prefs", prefs)

The `click_button` function is designed to click a button on a webpage identified by its CSS selector. The function will wait for the specified button to become clickable and then perform the click action.

#### Parameters:
- **driver**: 
  - Type: `selenium.webdriver.remote.webdriver.WebDriver`
  - Description: The Selenium WebDriver instance that is used to control the browser.
- **css_selector**: 
  - Type: `str`
  - Description: The CSS selector string that identifies the button on the webpage.

#### Functionality:
- The function waits for up to 10 seconds for the button to become clickable.
- Once the button is clickable, it performs a click action.

#### Exceptions:
- **TimeoutException**: 
  - Raised if the button does not become clickable within the specified timeout of 10 seconds.


In [5]:
def click_button(driver, css_selector):
    button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, css_selector))
    )
    button.click()

The `log_in` function automates the process of logging into a website by filling in the username and password fields, then clicking the login button.

#### Functionality:
- **Username and Password Input:**
  - The function waits for the username input field (`input[name="nameuser"]`) to be present, then fills it with the username retrieved from the environment variable `USERNAME`.
  - Similarly, it waits for the password input field (`input[name="password"]`) to be present, then fills it with the password retrieved from the environment variable `PASSWORD`.
  
- **Login Button Click:**
  - After entering the credentials, the function clicks the login button identified by the CSS selector `.orange-button`.

#### Parameters:
- **driver**:
  - Type: `selenium.webdriver.remote.webdriver.WebDriver`
  - Description: The Selenium WebDriver instance controlling the browser.

#### Dependencies:
- **Environment Variables:**
  - `USERNAME`: The environment variable storing the username.
  - `PASSWORD`: The environment variable storing the password.



In [6]:
def log_in(driver):
    input_field = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'input[name="nameuser"]'))
    )
    input_field.send_keys(os.getenv('USERNAME'))
    time.sleep(1)
    
    input_field = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'input[name="password"]'))
    )
    input_field.send_keys(os.getenv('PASSWORD'))
    time.sleep(1)
    
    click_button(driver, '.orange-button')

The `select_date` function is designed to automate the process of selecting and setting dates in input fields on a webpage.

#### Functionality:
- **Locate Date Input Fields:**
  - The function waits for all input fields with the CSS class `mx-input` to be present on the webpage.
  
- **Set Dates:**
  - The function iterates through the located input fields.
  - For each input field:
    - It clicks the field to focus on it.
    - It clears any existing content from the input field using both the `clear()` method and a JavaScript command.
    - It sends the corresponding date from the `date` list to the input field.

#### Parameters:
- **driver**:
  - Type: `selenium.webdriver.remote.webdriver.WebDriver`
  - Description: The Selenium WebDriver instance controlling the browser.

- **date**:
  - Type: `list`
  - Description: A list of date strings to be entered into the input fields. The list should have the same number of elements as the number of input fields on the page.



In [7]:
def select_date(driver, date):
    date_input = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'input.mx-input'))
    )
    for i, input_field in enumerate(date_input):
        input_field.click()
        input_field.clear()
        driver.execute_script("arguments[0].value = '';", input_field)
        input_field.send_keys(date[i])

The `select_channels` function automates the selection of specific TV channels from a list of checkboxes on a webpage. The function opens the channel selection menu, iterates through the available channels, and selects the ones specified in the `channels` list.

#### Functionality:
- **Open Channel Selection Menu:**
  - The function clicks a button identified by the CSS selector `button[data-v-2b71f047]` to open the channel selection menu.
  
- **Locate and Interact with Channel List:**
  - The function waits for the `<ol>` element containing the channel options (identified by `ol[data-v-2b71f047]`) to be present.
  - It retrieves all `<li>` elements within the `<ol>` and iterates through them.

- **Select Channels:**
  - For each `<li>` element (representing a channel category), the function:
    - Waits for the visibility of the `<li>` and ensures all checkboxes within it are located.
    - Clicks the `<li>` to reveal the checkboxes.
    - Locates the labels with the CSS class `.channelclass` that contain the checkboxes.
    - For each label:
      - Retrieves the checkbox and the corresponding channel name (contained in a `<span>` element).
      - Checks if the channel name is in the `channels` list.
      - If the channel is in the list and is not already selected, it clicks the checkbox to select it.

- **Finalize Selection:**
  - After selecting the channels, the function waits for all buttons with the CSS selector `button[data-v-2b71f047]` to be present.
  - It clicks the add button to confirm the selection.

#### Parameters:
- **driver**:
  - Type: `selenium.webdriver.remote.webdriver.WebDriver`
  - Description: The Selenium WebDriver instance controlling the browser.

- **channels**:
  - Type: `list`
  - Description: A list of channel names to be selected. The function will check the checkboxes for these channels if they are present on the page.



In [8]:
def select_channels(driver, channels):
    click_button(driver, 'button[data-v-2b71f047]')
    ol_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'ol[data-v-2b71f047]'))
    )
    li_elements = ol_element.find_elements(By.TAG_NAME, 'li')
    
    for li in li_elements:
        WebDriverWait(driver, 5).until(
            EC.visibility_of(li),
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'input[type="checkbox"]'))
        )
        li.click()
        WebDriverWait(driver, 5).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.channelclass'))
        )
        labels = driver.find_elements(By.CSS_SELECTOR, '.channelclass')
        for label in labels:
            # Get the checkbox input element and the text of the span
            checkbox = label.find_element(By.TAG_NAME, 'input')
            span_text = label.find_element(By.TAG_NAME, 'span').text
            # Check if the text is in the channels list
            if span_text in channels:
                # If not already selected, click the checkbox
                if not checkbox.is_selected():
                    checkbox.click()
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'button[data-v-2b71f047]'))
    )
    buttons = driver.find_elements(By.CSS_SELECTOR, 'button[data-v-2b71f047]')
    buttons[6].click()

The `select_days_of_the_week` function automates the process of selecting specific days of the week by interacting with a checkbox on a webpage. 

#### Functionality:
- **Open Days of the Week Dropdown:**
  - The function clicks a toggle button with the ID `#__BVID__86__BV_toggle_` to open the dropdown or section where the days of the week can be selected.

- **Locate and Interact with the Checkbox:**
  - The function waits for the checkbox with the ID `__BVID__88` to be present on the webpage.
  - Once the checkbox is located, it scrolls the checkbox into view using JavaScript to ensure it is visible on the screen.
  
- **Select the Checkbox:**
  - The function then attempts to click the checkbox to select the corresponding days of the week.
  - If a regular `click()` method fails, the function uses JavaScript to force the checkbox to be clicked.

#### Parameters:
- **driver**:
  - Type: `selenium.webdriver.remote.webdriver.WebDriver`
  - Description: The Selenium WebDriver instance controlling the browser.


In [11]:
def select_days_of_the_week(driver):
    click_button(driver, '#__BVID__86__BV_toggle_')
    checkbox = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, '__BVID__88'))
    )
    driver.execute_script("arguments[0].scrollIntoView(true);", checkbox)
    try:
        checkbox.click()
    except:
        driver.execute_script("arguments[0].click();", checkbox)

The `select_target_group` function automates the selection of specific target groups based on measures, age groups, and a consolidation option within a web form. This function selects the appropriate options from dropdown menus and adds the target group selections.

#### Functionality:
- **Open Target Group Selection Menu:**
  - The function begins by clicking a button identified by the CSS selector `button[data-v-62b88c45]` to reveal the target group selection section.

- **Select Consolidate Option:**
  - The function retrieves all dropdown elements associated with the selector `select[data-v-62b88c45]`.
  - It then locates the third dropdown in the list and selects the option that matches the `consolidate` value passed to the function.

- **Select Measure and Age Groups:**
  - The function interacts with the second dropdown to select measures specified in the `measures` list.
  - For each selected measure, the function iterates through the `age_groups` list and:
    - Selects the corresponding "from" and "to" values in the age range dropdowns (found as the 4th and 5th dropdowns in the list).
    - Clicks a button to add this combination of measure and age group.

- **Finalize Selection:**
  - After iterating through all measures and age groups, the function clicks the 14th button (index 13) in the list of buttons with the CSS selector `button[data-v-62b88c45]` to finalize the selection.

#### Parameters:
- **driver**:
  - Type: `selenium.webdriver.remote.webdriver.WebDriver`
  - Description: The Selenium WebDriver instance controlling the browser.

- **measures**:
  - Type: `list`
  - Description: A list of measure names to be selected from the dropdown menu (e.g., `['Rating %', 'Rating 000s', 'Share']`).

- **age_groups**:
  - Type: `list of dict`
  - Description: A list of dictionaries, where each dictionary contains 'from' and 'to' keys representing age ranges to be selected (e.g., `{'from': 15, 'to': 44}`).

- **consolidate**:
  - Type: `str`
  - Description: The text of the consolidate option to be selected from the third dropdown.


In [9]:
def select_target_group(driver, measures, age_groups, consolidate):
    click_button(driver, 'button[data-v-62b88c45]')
    select_elements = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'select[data-v-62b88c45]'))
    )
    select_consolidate = Select(select_elements[2])
    options = select_consolidate.options
    for option in options:
        option_text = option.text
        if option_text == consolidate:
            select_consolidate.select_by_visible_text(option_text)
    select_measure = Select(select_elements[1])
    options = select_measure.options
    for option in options:
        option_text = option.text
        if option_text in measures:
            select_measure.select_by_visible_text(option_text)
            for age_group in age_groups:
                select_from = Select(select_elements[3])
                select_from.select_by_value(str(age_group['from']))  
                select_to = Select(select_elements[4])
                select_to.select_by_value(str(age_group['to'])) 

                click_button(driver, '.btn.mr-2.btn-success.btn-sm')
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'button[data-v-62b88c45]'))
    )
    buttons = driver.find_elements(By.CSS_SELECTOR, 'button[data-v-62b88c45]')
    buttons[13].click()

### Selenium Script to Automate CSV Download Process

This script automates the process of logging into a website, selecting options, and downloading a CSV file using Selenium WebDriver.

#### Workflow:
1. **Setup WebDriver:**
    - The script initializes the Chrome WebDriver with specified options.
    - Environment variables are loaded using `load_dotenv()` to securely manage sensitive data like usernames and passwords.

2. **Navigate to Website:**
    - The WebDriver navigates to the specified URL (`https://tvanalys.mmsdata.se/`).

3. **Log In:**
    - The `log_in(driver)` function is called to input the username and password and submit the login form.

4. **Consent and Navigation:**
    - The script clicks a consent button (if present) and navigates to the "Programs" section of the website by clicking the appropriate link.

5. **Select Date:**
    - The `select_date(driver, date)` function is used to input the start and end dates into the form fields. The `date` list contains the dates to be entered.

6. **Select Channels:**
    - The `select_channels(driver, channels)` function is called to select TV channels based on the `channels` list.

7. **Select Days of the Week:**
    - The `select_days_of_the_week(driver)` function clicks a checkbox to select specific days of the week.

8. **Select Target Group:**
    - The `select_target_group(driver, measures, age_groups, consolidate)` function selects target group options based on provided measures, age groups, and a consolidation type.

9. **Download CSV:**
    - The script clicks the download button for the CSV file using `click_button(driver, '#download-csv-button')`.

10. **Wait for File Download:**
    - The script enters a loop, checking for the presence of files in the specified download folder. It breaks the loop when a file is detected.

11. **Close WebDriver:**
    - After the file is detected, the WebDriver is closed to end the session.

#### Parameters and Functions:
- **driver:** The WebDriver instance controlling the browser.
- **date:** A list of dates to be entered in the date input fields.
- **channels:** A list of channel names to be selected.
- **measures:** A list of measures to be selected from a dropdown.
- **age_groups:** A list of dictionaries specifying 'from' and 'to' values for age ranges.
- **consolidate:** The consolidation option to be selected.

#### Key Points:
- **`log_in(driver)`**: Handles the login process.
- **`select_date(driver, date)`**: Inputs the specified dates.
- **`select_channels(driver, channels)`**: Selects channels based on the list provided.
- **`select_days_of_the_week(driver)`**: Selects all days of the week.
- **`select_target_group(driver, measures, age_groups, consolidate)`**: Selects the specified target group and measures.
- **`click_button(driver, css_selector)`**: Utility function to click a button based on its CSS selector.
- **`os.listdir(download_folder)`**: Monitors the download folder for the presence of the downloaded CSV file.

#### Error Handling:
- The script is wrapped in a try-final


In [23]:
driver = webdriver.Chrome(options=chrome_options)
load_dotenv()

try:
    driver.get('https://tvanalys.mmsdata.se/')

    log_in(driver)
    click_button(driver, '.consent-button')
    click_button(driver, 'a[href="/programs"]')
    
    date = ['2023-11-01','2024-09-02']
    select_date(driver, date)
    
    channels = ['Kanal5','SVT1','SVT2','TV3','TV4']
    select_channels(driver, channels)
    
    select_days_of_the_week(driver)
    
    measures = ['Rating %', 'Rating 000s', 'Share']
    age_groups = [{'from':3,'to':99},{'from':15,'to':64},{'from':15,'to':44}]
    consolidate = 'OV'
    select_target_group(driver, measures, age_groups, consolidate)
    
    click_button(driver, '#download-csv-button')
    
    initial_files = set(os.listdir(download_folder))
    while True:
        current_files = set(os.listdir(download_folder))
        new_files = current_files - initial_files
        if new_files:
            # New files have appeared in the folder
            break
        time.sleep(2)

finally:
    driver.quit()

In [20]:
import os
import pandas as pd
import csv

The `remove_summary` function processes a CSV file to remove any summary or introductory lines, retaining only the rows following a specified header line. This is useful for cleaning up CSV files that contain extraneous information before the actual data begins.

#### Functionality:
- **Identify and Remove Summary Lines:**
  - The function reads through the CSV file and searches for a specific header line.
  - Once the specified header line is found, the function starts writing all subsequent lines to a new temporary file.

- **Create a Cleaned CSV File:**
  - The function writes the header line and all subsequent rows to a temporary file.
  - After processing, the original file is replaced with the temporary file to reflect the cleaned content.

#### Parameters:
- **file_path**:
  - Type: `str`
  - Description: The path to the CSV file that needs to be cleaned.

- **header_line**:
  - Type: `str` (Optional)
  - Description: The specific header line to search for in the CSV file. This line will be written as the header in the cleaned file, and all lines following it will be retained. If not specified, the default header provided will be used.



In [18]:
def remove_summary(file_path, header_line='Datum;Starttid;Sluttid;Längd;Kanalnamn;Titel;Titelkod;Genre1;Genre2;Genre3;Rating % A3-99 [OV];Rating % A15-64 [OV];Rating % A15-44 [OV];Rating 000s A3-99 [OV];Rating 000s A15-64 [OV];Rating 000s A15-44 [OV];Share A3-99 [OV];Share A15-64 [OV];Share A15-44 [OV]'):
    header_found = False
    temp_file_path = file_path + '.tmp'

    with open(file_path, 'r', newline='') as infile, open(temp_file_path, 'w', newline='') as tempfile:
        reader = csv.reader(infile, delimiter=';')
        writer = csv.writer(tempfile, delimiter=';')
        
        for row in reader:
            current_line = ';'.join(row)
            
            if current_line == header_line:
                header_found = True
                writer.writerow(row)  # Write the header line to the temporary file
                continue
            
            if header_found:
                writer.writerow(row)
    os.replace(temp_file_path, file_path)



This code snippet performs the following tasks:

1. **Identify CSV Files:**
   - Sets the folder path to `'files'`.
   - Filters and retrieves a list of CSV files in the directory.

2. **Select the Most Recent CSV File:**
   - Constructs full paths for each CSV file in the directory.
   - Gets the modification time for each file.
   - Identifies the most recent file based on the latest modification time.

3. **Convert into XLSX format:**
   - Removes summary lines from the most recent CSV file using the `remove_summary` function.
   - Reads the cleaned CSV file into a pandas DataFrame with a semicolon (`;`) as the delimiter.
   - Converts the DataFrame to an Excel file (`.xlsx` format) and saves it in the same directory with the same name as the original CSV file.



In [24]:
# Assuming the folder is 'files'
folder_path = 'files'
files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

if files:
    # Get the path of the most recent CSV file
    file_paths = [os.path.join(folder_path, f) for f in files]
    file_mod_times = [os.path.getmtime(path) for path in file_paths]
    latest_file_index = file_mod_times.index(max(file_mod_times))
    file = file_paths[latest_file_index]
    # Remove the summary
    remove_summary(file)
    # Convert csv to xlsx
    df = pd.read_csv(file, sep=';')
    xlsx_path = os.path.join(folder_path, os.path.splitext(os.path.basename(file))[0] + '.xlsx')
    df.to_excel(xlsx_path, index=False)